In [1]:
# GWU Bootcamp Module 6 Web APIs and Pandas Data Frames
# Module 6 Challenge - Deliverable # 1- Retrieve Weather Data and Export to CSV
# Student name: Christopher Mastrangelo

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

# very important to include this module - this is what was causing error
import requests

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
# Import the API key.  will need to copy the config.py file to this folder
from config import weather_api_key

In [3]:
# Create a set of random latitude and longitude combinations.
# I think -90 to 90 is too likely to pick a non populated area
# Even using -60 to 70 you are going to get some extremely remote locations, most over water 
# start with 20 until its working then scale up to 200 then to 2000 GPS pairs for the challenge

# For the current run - keep cities in USA and Canada including Alaska and Hawaii- just to show I can 
maxsize = 675
lats = np.random.uniform(low=24.000, high=72.000, size=maxsize)
lngs = np.random.uniform(low=-165.000, high=-65.000, size=maxsize)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
# coordinates

In [5]:
# This is how we build the unique list of cities. only the name field is unique.  
# question - what if there is a city name like springfield which is located in more than one state?
# that is beyond the scope of this assignment but I would like to verify if the lat/long matches the original used

# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
# cities

# I said it before and I will say it again- none of these places are places any traveler would actually go
# they happen to be islands or places along the coast because random pins dropped over the ocean end up in Hawaii
# or Capetown South Africa as the nearest city on the coast
# make sure to add this comment to the README file when submitting the challenge

246

In [6]:
# Starting URL for Weather Map API Call - do this outside the loop and do not PRINT IT
# Be careful how many times you run this in case there is a limit on the number of free uses for API Key

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an empty list to hold the weather data.
city_data = []

# for city in cities
for i, city in enumerate(cities):
    
    # Create an endpoint URL for a city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # copies this code right out of the WeatherPy notebook - we just have to add "description"
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data. Put them in the order we need so we don't have to reorder columns later
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        
        # next line is the new line added
        city_description = city_weather["weather"][0]['description']

        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        if (city_country not in ["CA","US"]):
            print("Skipping ", city.title(), city_country, city_lat, city_lng)
        else:
            city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})
                          # removing date for the challenge was >>> "Date": city_date})
        # show which cities are in Canada, Alaska or Hawaii
        if ((city_lat > 49.75) or (city_lng < -124.6)):
            print(city.title(), city_country, city_lat, city_lng)
            
# If an error is experienced, skip the city. using silent diagnostics instead of print statements 
    except:
        print("City: " + city + " not found. Skipping...")
        pass

# If we process more than 200 cities we should batch them in groups of 50 like the we did in the WeatherPy Notebook
# If we demonstrate using 200 cities in USA and Canada that should prove the code is scalable to 2000 locations
# As indicated above I enhanced random code to only select cities in USA including ALaska Hawaii or Canada


City: grand centre not found. Skipping...
City: attawapiskat not found. Skipping...
Prince Rupert CA 54.3161 -130.3201
Barrow US 71.2906 -156.7887
Port-Cartier CA 50.0334 -66.8654
Skipping  Constitucion CL -35.3333 -72.4167
Fairbanks US 64.8378 -147.7164
Pangnirtung CA 66.1451 -65.7125
Hilo US 19.7297 -155.09
Hay River CA 60.8156 -115.7999
Kodiak US 57.79 -152.4072
Skipping  General Cepeda MX 25.3833 -101.45
Kapaa US 22.0752 -159.319
Thompson CA 55.7435 -97.8558
Clyde River CA 70.4692 -68.5914
Fort Nelson CA 58.8053 -122.7002
Skipping  Eldorado BR -23.7869 -54.2836
Homer US 59.6425 -151.5483
Skipping  Guerrero Negro MX 27.9769 -114.0611
Kahului US 20.8947 -156.47
Flin Flon CA 54.7682 -101.865
Skipping  Winchester GB 51.0651 -1.3187
Winchester GB 51.0651 -1.3187
Ketchikan US 55.3422 -131.6461
Yellowknife CA 62.456 -114.3525
Sioux Lookout CA 50.1001 -91.917
Skipping  Kincardine GB 56.0667 -3.7167
Kincardine GB 56.0667 -3.7167
Chapais CA 49.7834 -74.8492
City: fort saint john not found. S

In [7]:
# DEBUG - this cell only needs to be run when used to parse the JSON output
# used this cell to traverse the JSON and peel the layers off the onion to get the syntax for each item
# do these steps outside the try/catch so we can see what is going on 

city = 'westminster'
city_url = url + "&q=" + city.replace(" ","+")
        
city_weather = requests.get(city_url).json()
        
# print(city_weather)
# if the status code will not print maybe that is what is causing the catch

city_lat = city_weather["coord"]["lat"]
city_lng = city_weather["coord"]["lon"]
city_max_temp = city_weather["main"]["temp_max"]
city_humidity = city_weather["main"]["humidity"]

# next line is the new line added
city_description = city_weather["weather"][0]['description']

# city_clouds = city_weather["clouds"]["all"]
# city_wind = city_weather["wind"]["speed"]
# city_country = city_weather["sys"]["country"]

    
print(city_description)
city_weather["weather"][0]['description']


broken clouds


'broken clouds'

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(20)


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Palmer,US,42.1584,-72.3287,74.01,73,20,5.75,few clouds
1,Houma,US,29.5958,-90.7195,84.97,81,22,5.12,few clouds
2,Roberval,CA,48.5168,-72.2324,72.23,59,100,12.64,overcast clouds
3,Poplar Bluff,US,36.7570,-90.3929,76.78,73,1,4.61,clear sky
4,Prince Rupert,CA,54.3161,-130.3201,60.89,82,90,6.91,light rain
5,Barrow,US,71.2906,-156.7887,39.22,87,90,12.66,overcast clouds
6,Fortuna,US,40.5982,-124.1573,81.19,60,1,8.01,clear sky
7,Port-Cartier,CA,50.0334,-66.8654,60.75,80,53,5.06,broken clouds
8,Fairbanks,US,64.8378,-147.7164,67.30,79,90,1.01,light rain
9,Pangnirtung,CA,66.1451,-65.7125,39.24,91,100,1.34,overcast clouds


In [9]:
# Create the output file (CSV).
output_data_file = "./WeatherPy_Database.csv"
# Export the City_Data into a CSV. SAme folder that we are in which should be World_Weather

# ONLY RE-ENABLE THIS LIINE IF YOU WANT TO SAVE ANOTHER COPY OF THE OUTPUT FILE 
# Header default is TRUE so no need to include the column headers except the label for the first column
city_data_df.to_csv(output_data_file, index_label="City_ID")

# Looks like we are ready to submit Deliverable #1 for Module 6 Challenge date 2021-08-06